# Connect Franka

In [1]:
# Panda hostname/IP and Desk login information of your robot
hostname = '172.16.0.2'
username = 'admin'
password = '******'

# panda-py is chatty, activate information log level
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
import panda_py
from panda_py import libfranka

panda = panda_py.Panda(hostname)
gripper = libfranka.Gripper(hostname)
panda.move_to_start()

In [3]:
from panda_py import constants
from scipy.spatial.transform import Rotation as R
import numpy as np
import matplotlib.pyplot as plt
import os
import time

# Shoot from Different Views

In [4]:
# read shooting pose(7 dof joint position in one line) from file
joint_position_list = np.loadtxt("shooting_pose_tabletop.txt", delimiter=',')

In [ ]:
import sys
parent_dir = os.path.dirname(os.getcwd())
parent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
from realsense.realsense import Camera


base_dir = '/home/annabella/Documents/jima_ws/grasp_splats_old/scene_data/release_video'
os.makedirs(f'{base_dir}/images', exist_ok=True)
os.makedirs(f'{base_dir}/depth', exist_ok=True)
os.makedirs(f'{base_dir}/poses', exist_ok=True)

device_serial = '145422070656'

rgb_resolution = (1280, 720)  # RGB resolution (width, height)
depth_resolution = (1280, 720)  # Depth resolution (width, height)
camera = Camera(device_serial, rgb_resolution, depth_resolution)
camera.start()

rgb_intrinsics, rgb_coeffs, depth_intrinsics, depth_coeffs = camera.get_intrinsics_raw()
depth_scale = camera.get_depth_scale()

print(f"RGB Intrinsics: {rgb_intrinsics}")
print(f"RGB Distortion Coefficients: {rgb_coeffs}")
rgb_intrinsics_path = f'{base_dir}/rgb_intrinsics.npz'
np.savez(rgb_intrinsics_path, fx=rgb_intrinsics.fx, fy=rgb_intrinsics.fy, ppx=rgb_intrinsics.ppx, ppy=rgb_intrinsics.ppy, coeffs=rgb_intrinsics.coeffs)

print(f"Depth Scale: {depth_scale}")
print(f"Depth Intrinsics: {depth_intrinsics}")
print(f"Depth Distortion Coefficients: {depth_coeffs}")
depth_intrinsics_path = f'{base_dir}/depth_intrinsics.npz'
np.savez(depth_intrinsics_path, fx=depth_intrinsics.fx, fy=depth_intrinsics.fy, ppx=depth_intrinsics.ppx, ppy=depth_intrinsics.ppy, coeffs=depth_intrinsics.coeffs, depth_scale=depth_scale)

# drop the first few frames to allow the camera to warm up
_, _ = camera.shoot()  
time.sleep(2)
panda.move_to_start()

for i in range(joint_position_list.shape[0]):
    joint_position = joint_position_list[i]
    panda.move_to_joint_position(joint_position)
    pose = panda.get_pose()

    rgb_image, depth_image = camera.shoot()
    depth_image = depth_image * depth_scale
    rgb_filename = f'{base_dir}/images/{i}.png'
    depth_filename = f'{base_dir}/depth/{i}.npy'
    plt.imsave(rgb_filename, rgb_image)
    np.save(depth_filename, depth_image)
    print(f"Saved {rgb_filename}")
    print(f"Saved {depth_filename}")

    def save_pose(pose, base_dir, frame_num):
        """Save the robot arm's pose to a file."""
        pose_filename = f'{base_dir}/poses/{frame_num}.npy'
        np.save(pose_filename, pose)
        print(f"Saved pose to {pose_filename}")

    save_pose(pose, base_dir, i)